#### Implementing regularization 

In [1]:
import os
import re
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import utils
import keras
import imageio 
from PIL import Image 

Using TensorFlow backend.


In [2]:
# Reading the data
train = pd.read_csv('age_detection_train/train.csv')

# Image resizing of test data into single numpy array
temp = []
for img_name in train.ID:
    img_path = os.path.join('age_detection_train/Train', img_name)
    img = imageio.imread(img_path)
    img = np.array(Image.fromarray(img).resize((32, 32))).astype('float32')    
    temp.append(img)

train_x = np.stack(temp)

# Normalizing the images
train_x = train_x / 255.

# Encoding the categorical variable to numeric
lb = LabelEncoder()
train_y = lb.fit_transform(train.Class)
train_y = utils.np_utils.to_categorical(train_y)

In [5]:
# Specifying all the parameters we will be using in our network
input_num_units = (32, 32, 3)
hidden_num_units = 500
output_num_units = 3

epochs = 21
batch_size = 512

In [6]:
def regularization(list_of_regs, reg_names):
    for reg_init in range(len(list_of_regs)):        
        # Writing structure of model    
        model = Sequential([
          InputLayer(input_shape=input_num_units),
          Flatten(),
          Dense(units=hidden_num_units, kernel_regularizer=list_of_regs[reg_init], activation='relu'),
          Dense(units=output_num_units, activation='softmax'),
        ])
        # Defining parameters like optmizer, loss function and evaluating metric
        model.compile(loss='categorical_crossentropy', # 
                      optimizer=keras.optimizers.Adam(), # Learning rate and momentum can be passed inside optimizer
                      metrics=['accuracy'])

        # Traning the model and writing log file for TensorBoard
        
        logdir = r'regularization\\' + reg_names[reg_init] # Each log file needs to be written in a distinct directory. (Mandatory)        

        # To store tensorboard graph logs
        cb = keras.callbacks.TensorBoard(log_dir=logdir, write_graph=False) 
        
        print('Model building using ' + reg_names[reg_init] + ' Regularization')
        history = model.fit(train_x, train_y, epochs=epochs, 
                           validation_split=0.2,
                           callbacks=[cb])
        print('Model built successfully.')
        print('')

In [7]:
# Value of lambda can be calculated using cross-validation
list_of_regs = [keras.regularizers.l2(), keras.regularizers.l1(), keras.regularizers.l1_l2(l1=0.1, l2=0.8)] 
reg_names = ['Ridge.L2', 'Lasso.L1', 'Elastic.Net.L1.L2']
regularization(list_of_regs, reg_names)

Model building using Ridge.L2 Regularization
Train on 15924 samples, validate on 3982 samples
Epoch 1/21
15924/15924 [==============================] - 58s 4ms/step - loss: 2.0490 - acc: 0.5653 - val_loss: 0.9921 - val_acc: 0.6002
Epoch 2/21
15924/15924 [==============================] - 59s 4ms/step - loss: 0.9313 - acc: 0.5857 - val_loss: 0.8806 - val_acc: 0.6087
Epoch 3/21
15924/15924 [==============================] - 52s 3ms/step - loss: 0.8994 - acc: 0.5935 - val_loss: 0.8688 - val_acc: 0.6195
Epoch 4/21
15924/15924 [==============================] - 59s 4ms/step - loss: 0.8897 - acc: 0.6031 - val_loss: 0.8850 - val_acc: 0.6205
Epoch 5/21
15924/15924 [==============================] - 61s 4ms/step - loss: 0.8876 - acc: 0.6033 - val_loss: 0.8550 - val_acc: 0.6293
Epoch 6/21
15924/15924 [==============================] - 66s 4ms/step - loss: 0.8812 - acc: 0.6112 - val_loss: 0.8734 - val_acc: 0.6251
Epoch 7/21
15924/15924 [==============================] - 69s 4ms/step - loss: 0.880

Epoch 16/21
15924/15924 [==============================] - 61s 4ms/step - loss: 18.1734 - acc: 0.5425 - val_loss: 18.2603 - val_acc: 0.5439
Epoch 17/21
15924/15924 [==============================] - 61s 4ms/step - loss: 18.1859 - acc: 0.5425 - val_loss: 18.3350 - val_acc: 0.5439
Epoch 18/21
15924/15924 [==============================] - 61s 4ms/step - loss: 18.1437 - acc: 0.5425 - val_loss: 18.1400 - val_acc: 0.5439
Epoch 19/21
15924/15924 [==============================] - 64s 4ms/step - loss: 18.1578 - acc: 0.5425 - val_loss: 18.4712 - val_acc: 0.5439
Epoch 20/21
15924/15924 [==============================] - 69s 4ms/step - loss: 18.1338 - acc: 0.5425 - val_loss: 18.0164 - val_acc: 0.5439
Epoch 21/21
15924/15924 [==============================] - 71s 4ms/step - loss: 18.1060 - acc: 0.5425 - val_loss: 18.1152 - val_acc: 0.5439
Model built successfully.



#### Open tensorboard in `regularization` directory